In [ ]:
#SOFTVOTE : 해당 노트북에서 사용했던 게 아니라, 다른 노트북에서 사용했던 거라서 IMPORT가 없음.
#원래는 Code.ipynb에 존재.

def multi_model_ensemble_test (models, weights, dataloader, device) :
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    for model in models :
        model.eval()
    file_name_list = []
    #file_list_
    preds_array = np.empty((0, size*size), dtype=np.long)
    with torch.no_grad():
        for step, (imgs, image_infos) in tqdm(enumerate(test_loader)):
            x = torch.stack(imgs).to(device)
            # inference (512 x 512)
            outs = models[0](x) * weights[0]
            isFirst = True
            for model, weight in zip(models, weights) :
                if isFirst :
                    isFirst = False
                else :
                    outs += model(x) * weight
            if outs.shape[0] == 1 : oms = torch.argmax(outs.squeeze(), dim = 0).detach().cpu().numpy()
            else : oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)
            oms = np.array(temp_mask)
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    return file_names, preds_array
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
models = list()
weights = list()
dir_name = "./TOP3/"
for model_name in ["dv3p_16batch_adamp_512_cos.pt",\
                  "dv3p_16batch_adamp_512_full.pt",\
                  "dv3p_16batch_adamp_512.pt"] :
    model_name = dir_name + model_name
    model = smp.DeepLabV3Plus(
                encoder_name='resnext101_32x4d',
                encoder_weights = 'ssl',
                classes = class_nums).to(device)
    model.load_state_dict(torch.load(model_name))
    models.append(model)
    weights.append(1)
#가중치 추가
weights[0] += 0.1
model.load_state_dict(torch.load(model_name))
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)
# submission.csv로 저장
save_name = "ensemble"
submission.to_csv("./submission/"+save_name+".csv", index=False)

In [84]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

def hard_vote_models (csv_files, best_model) :
    out_submission = pd.DataFrame(columns=["image_id", "PredictionString"])
    out_submission["image_id"] = csv_files[0]["image_id"]
    #print(out_submission.head())
    
    whole_prediction = list()
    for i in tqdm(range(len(csv_files[0]))) :
        predict_arr = np.zeros(256*256)
        predict_arr_tmp = np.zeros((256*256, 12))
        
        for csv_file in csv_files :
            #print(csv_file.iloc[i])
            predict = csv_file.iloc[i]["PredictionString"]
            predict = predict.split()
            
            for j, pixel_predict in enumerate(predict) :
                predict_arr_tmp[j][eval(pixel_predict)] += 1
            
        for j, _ in enumerate(predict_arr_tmp) :
            max_index = np.argmax(predict_arr_tmp[j])
            max_index_size = np.where(predict_arr_tmp[j] == predict_arr_tmp[j][max_index])
            #print(max_index_size)
            if len(max_index_size) == 1 :
                predict_arr[j] = max_index
            elif csv_files[best_model].iloc[i].split()[j] in max_index_size :
                predict_arr[j] = csv_files[best_model].iloc[i].split()[j]
            else :
                predict_arr[j] = random.choice(max_index_size)
                
        whole_prediction.append(' '.join(str(int(e)) for e in predict_arr.tolist()))
    
    out_submission["PredictionString"] = whole_prediction
    return out_submission
        
sub_dir = "submission/"
files = ["output (0).csv", \
         "output (1).csv", \
         "output (2).csv"]
         #"output (3).csv", \
         #"output (4).csv"]

for i, file in enumerate(files) :
    files[i] = pd.read_csv(sub_dir + file)
    
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0) 
save_name = "hardvote_ensemble"
submission.to_csv("./submission/"+save_name+".csv", index=False)

  8%|▊         | 63/837 [01:38<20:09,  1.56s/it]


KeyboardInterrupt: 

In [12]:
import pandas as pd
import numpy as np

index = 5
sub = pd.read_csv('ensemble_result/soft_vote_1_1.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_1_1')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_1_2.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_1_2')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_1_3.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_1_3')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_1_4.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_1_4')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_1_5.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_1_5')
print(arr)

print('-------------------------------')

index = 7
sub = pd.read_csv('ensemble_result/soft_vote_2_1.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_2_1')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_2_2.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_2_2')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_2_3.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_2_3')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_2_4.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_2_4')
print(arr)

index = 452
sub = pd.read_csv('ensemble_result/soft_vote_3_1.csv')
s = sub.iloc[index]["PredictionString"]

print('-------------------------------')

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_3_1')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_3_2.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_3_2')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_3_3.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_3_3')
print(arr)

sub = pd.read_csv('ensemble_result/soft_vote_3_4.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_3_4')
print(arr)

softvote_1_1
[6.5535e+04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00]
softvote_1_2
[65536.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.]
softvote_1_3
[65536.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.]
softvote_1_4
[6.5534e+04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 2.0000e+00 0.0000e+00 0.0000e+00]
softvote_1_5
[6.5531e+04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 5.0000e+00 0.0000e+00 0.0000e+00]
-------------------------------
softvote_2_1
[62431.     0.   174.     0.     0.     0.     0.  2931.     0.     0.
     0.     0.]
softvote_2_2
[62431.     0.   174.     0.     0.     0.     0.  2931.     0.     0.
     0.     0.]
softvote_2_3
[62440.     0.   177.     0.     0.     0.     0.  2919.     0.     0.
     0.     0.]
softvote_2_4
[6.2428e+04